<a href="https://colab.research.google.com/github/GallaDevanshi/NLP-Review-Analysis/blob/main/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMDB data analysis using logistic regression

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD as TSVD
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict

import spacy
from spacy.lang.en.stop_words import STOP_WORDS as stopwords

In [ ]:
data=pd.read_csv('/content/IMDB Dataset.csv')

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.shape

(50000, 2)

In [ ]:
train = data.iloc[:25000]
test = data.iloc[25000:]

In [ ]:
train.shape, test.shape

((25000, 2), (25000, 2))

In [ ]:
train.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
train['review'] = train['review'].apply(lambda x: str(x).lower())

<ipython-input-56-af4ce04224a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = train['review'].apply(lambda x: str(x).lower())


In [ ]:
train.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 15.3 MB/s eta 0:00:00


In [ ]:
import contractions

In [ ]:
contractions_dict = contractions.contractions_dict
contractions_dict


{"I'm": 'I am',
 "I'm'a": 'I am about to',
 "I'm'o": 'I am going to',
 "I've": 'I have',
 "I'll": 'I will',
 "I'll've": 'I will have',
 "I'd": 'I would',
 "I'd've": 'I would have',
 'Whatcha': 'What are you',
 "amn't": 'am not',
 "ain't": 'are not',
 "aren't": 'are not',
 "'cause": 'because',
 "can't": 'cannot',
 "can't've": 'cannot have',
 "could've": 'could have',
 "couldn't": 'could not',
 "couldn't've": 'could not have',
 "daren't": 'dare not',
 "daresn't": 'dare not',
 "dasn't": 'dare not',
 "didn't": 'did not',
 'didn’t': 'did not',
 "don't": 'do not',
 'don’t': 'do not',
 "doesn't": 'does not',
 "e'er": 'ever',
 "everyone's": 'everyone is',
 'finna': 'fixing to',
 'gimme': 'give me',
 "gon't": 'go not',
 'gonna': 'going to',
 'gotta': 'got to',
 "hadn't": 'had not',
 "hadn't've": 'had not have',
 "hasn't": 'has not',
 "haven't": 'have not',
 "he've": 'he have',
 "he's": 'he is',
 "he'll": 'he will',
 "he'll've": 'he will have',
 "he'd": 'he would',
 "he'd've": 'he would have',
 

In [ ]:
def contraction_expansion(x):

    if type(x) is str:

        for key in contractions_dict:

            value = contractions_dict[key]

            x = x.replace(key, value)

        return x

    else:

        return x

In [ ]:
train['review'] = train['review'].apply(lambda x: contraction_expansion(x))

<ipython-input-62-1ad10dcbaf8e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = train['review'].apply(lambda x: contraction_expansion(x))


In [ ]:
train.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there is a family where a little boy...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [ ]:
def remove_emails(x):

    email_pattern = re.compile(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)")

    return re.sub(email_pattern, '', x)

In [ ]:
train['review'] = train['review'].apply(lambda x:remove_emails(x))

<ipython-input-65-612e19e984dc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = train['review'].apply(lambda x:remove_emails(x))


In [ ]:
train.sample(10)

,review,sentiment
13411,the theme is controversial and the depiction o...,positive
5225,"i saw this movie a time ago, because some of m...",negative
4725,this waste of time is a completely unnecessary...,negative
12831,this movie provided nothing new or worthwhile....,negative
8732,this movie is a little slow in the the beginni...,positive
22881,"the supernatural, vengeful police officer is b...",negative
14018,recently shown on cable tv the movie opens wit...,positive
24411,"that this film flopped at the box office, and ...",positive
10143,after growing up in the gritty streets of detr...,positive
3651,it is hard to describe this film and one wants...,negative


In [ ]:
train['review'] = train['review'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text().strip())

<ipython-input-67-1e7f389bc0f1>:1: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  train['review'] = train['review'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text().strip())
<ipython-input-67-1e7f389bc0f1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = train['review'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text().strip())


In [ ]:
train.iloc[6005][0]

'pretty.pretty actresses and actors. pretty bad script. pretty frequent "let us strip to our undies" scenes. pretty fair f/x. pretty jarring location decisions (the college dorm room looks like a high-end hotel room - probably because it was shot at a hotel). pretty bland storyline. pretty awful dialog. pretty locations. pretty annoying editing, unless you like the music video flash-cut style.this one is not a guilty pleasure - this is more an embarrassing one. if you must watch this, pick a good dance/techno album and turn the sound off on the movie - you will see the pretty people in their pretty black undies, and probably follow the story just fine.the cast may be able to act - i doubt that anyone could look skilled given the lines/plot that they had to deal with.'

In [ ]:
train.sample(5)

,review,sentiment
1506,now this is what i'd call a good horror. with ...,positive
158,the story is very trustworthy and powerful. th...,negative
23460,"okay, i'll admit that if i did not have kids, ...",positive
9183,i've watched this film a few times and i never...,negative
15546,"in ""red letters"", coyote is at the vortex of a...",negative


In [ ]:
def RemoveSpecialChars(x):

    x = re.sub(r'[^\w ]+', "", x)
    x = ' '.join(x.split())
    return x

In [ ]:
train['review'] = train['review'].apply(lambda x: RemoveSpecialChars(x))

<ipython-input-72-f1acf1517d58>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = train['review'].apply(lambda x: RemoveSpecialChars(x))


In [ ]:
train.sample(5)

,review,sentiment
14211,all i can say about the necromaniacschizophren...,positive
13259,caddyshack 2 has a dreadful reputation due onl...,positive
7557,stalingrad is a terrific movie well acted and ...,positive
15753,while trying to build a major mall or complex ...,negative
10827,i had always been a big lynda carterwonder wom...,negative


In [ ]:
train.iloc[6005][0]

'prettypretty actresses and actors pretty bad script pretty frequent let us strip to our undies scenes pretty fair fx pretty jarring location decisions the college dorm room looks like a highend hotel room probably because it was shot at a hotel pretty bland storyline pretty awful dialog pretty locations pretty annoying editing unless you like the music video flashcut stylethis one is not a guilty pleasure this is more an embarrassing one if you must watch this pick a good dancetechno album and turn the sound off on the movie you will see the pretty people in their pretty black undies and probably follow the story just finethe cast may be able to act i doubt that anyone could look skilled given the linesplot that they had to deal with'

In [ ]:
def lemme(x):

    x = str(x)
    x_list = []
    doc = nlp(x)

    for token in doc:
        lemma = token.lemma_

        if lemma == '-PRON-' or lemma == 'be':
            lemma = token.text

        x_list.append(lemma)

    return ' '.join(x_list)


In [ ]:
%%time
train['review'] = train['review'].apply(lambda x: lemme(x))

CPU times: user 20min 27s, sys: 2.39 s, total: 20min 30s
Wall time: 20min 50s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
train.sample(5)

,review,sentiment
23146,I enter my first comment on this film almost f...,positive
15900,year ago ma2412 the feature film try to be som...,negative
8094,have you ever see a movie make up entirely of ...,negative
16119,frankly I do not think there was anything humo...,negative
7868,this movie was a pleasant surprise because I d...,positive


In [ ]:
stopwords

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'fron

In [ ]:
len(stopwords)


326

In [ ]:
def RemoveStopWords(x):

    return ' '.join([word for word in x.split() if word not in stopwords])

In [ ]:
x = train.iloc[6005][0]


In [ ]:
print(x)
print()
print("length of x: ",len(x))


prettypretty actress and actor pretty bad script pretty frequent let we strip to our undie scene pretty fair fx pretty jarring location decision the college dorm room look like a highend hotel room probably because it was shoot at a hotel pretty bland storyline pretty awful dialog pretty location pretty annoying editing unless you like the music video flashcut stylethis one is not a guilty pleasure this is more an embarrassing one if you must watch this pick a good dancetechno album and turn the sound off on the movie you will see the pretty people in their pretty black undie and probably follow the story just finethe cast may be able to act I doubt that anyone could look skilled give the linesplot that they have to deal with

length of x:  735


In [ ]:
x1 = RemoveStopWords(x)
x1

'prettypretty actress actor pretty bad script pretty frequent let strip undie scene pretty fair fx pretty jarring location decision college dorm room look like highend hotel room probably shoot hotel pretty bland storyline pretty awful dialog pretty location pretty annoying editing like music video flashcut stylethis guilty pleasure embarrassing watch pick good dancetechno album turn sound movie pretty people pretty black undie probably follow story finethe cast able act I doubt look skilled linesplot deal'

In [ ]:
len(x1)

510

In [ ]:
%%time

train['review'] = train['review'].apply(lambda x: RemoveStopWords(x))

CPU times: user 1.17 s, sys: 7.97 ms, total: 1.18 s
Wall time: 1.2 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
train.sample(5)

,review,sentiment
6200,think bad movie world think person design cove...,negative
14082,opinion movie course badi expect horrible crap...,negative
2514,define scene movie fat guy quitsbut evil docto...,negative
24133,subject matter deserve script final result mov...,negative
13765,I usually like lawrence australia I stand I vi...,negative


In [ ]:
text = ' '.join(train['review'])


In [ ]:
len(text)

17520443

In [ ]:
text_series = pd.Series(text.split())

In [ ]:
freq_comm = text_series.value_counts()

In [ ]:
freq_comm

I                    90117
movie                49624
film                 45914
like                 21488
good                 20105
                     ...  
allhell                  1
lamenessseriously        1
x1                       1
xraye                    1
tvpersonality            1
Name: count, Length: 129591, dtype: int64

In [ ]:
rare_words = freq_comm[-82000:-1]
'rockumentarie' in rare_words

True

In [ ]:
rare_words

communityas          2
consummately         2
nozzle               2
rca                  2
ich                  2
                    ..
buffalothe           1
allhell              1
lamenessseriously    1
x1                   1
xraye                1
Name: count, Length: 81999, dtype: int64

In [ ]:
train['review'] = train['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in rare_words]))

<ipython-input-100-9ea04dacf819>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = train['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in rare_words]))


In [ ]:
train['review'].sample(5)


5503     enjoy start 2 hour movie unconnected burst unw...
4615     mishima life chapter visually stunning product...
18095    I early preview movie I hope time edit way imp...
13720    I m christian I watch pretty christian movie b...
20452    simply terrible use actual actor look stop sto...
Name: review, dtype: object

In [ ]:
train['sentiment'].value_counts()

sentiment
negative    12526
positive    12474
Name: count, dtype: int64

In [ ]:
X = train['review']
y = train['sentiment']

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
X = tfidf.fit_transform(X)

In [ ]:
X.shape

(25000, 47553)

In [ ]:
X

<25000x47553 sparse matrix of type '<class 'numpy.float64'>'
	with 1933813 stored elements in Compressed Sparse Row format>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4, stratify = y)

In [ ]:
X_train.shape, X_test.shape

((20000, 47553), (5000, 47553))

In [ ]:
%%time
#tsvd = TSVD(n_components=10000, random_state=4)
#X_train_tsvd = tsvd.fit_transform(X_train)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs


In [ ]:
%%time

#scores = cross_val_score(clf_svc, X_train, y_train, cv=6, n_jobs=-1)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 20 µs


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf_lr = LogisticRegression()

In [ ]:
X_train

<20000x47553 sparse matrix of type '<class 'numpy.float64'>'
	with 1548206 stored elements in Compressed Sparse Row format>

In [ ]:
%%time

scores = cross_val_score(clf_lr, X_train, y_train, cv=10, n_jobs=4)

CPU times: user 265 ms, sys: 311 ms, total: 576 ms
Wall time: 14.1 s


In [ ]:
scores

array([0.877 , 0.8815, 0.8795, 0.8755, 0.8915, 0.894 , 0.877 , 0.8745,
       0.8705, 0.873 ])

In [ ]:
scores.mean()

0.8793999999999998

In [ ]:
clf_lr.fit(X_train, y_train)

LogisticRegression()

In [ ]:
y_test_pred = clf_lr.predict(X_test)


In [ ]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

    negative       0.89      0.85      0.87      2505
    positive       0.86      0.90      0.88      2495

    accuracy                           0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000



In [ ]:
confusion_matrix(y_test, y_test_pred)

array([[2137,  368],
       [ 252, 2243]])

In [ ]:
clf_lr.predict(tfidf.transform(['American Psycho deserved an Oscar, they were robbed']))

array(['positive'], dtype=object)

In [ ]:
y_real_pred = clf_lr.predict(tfidf.transform(test['review']))


In [ ]:
print(classification_report(test['sentiment'], y_real_pred))

              precision    recall  f1-score   support

    negative       0.89      0.83      0.86     12474
    positive       0.84      0.89      0.87     12526

    accuracy                           0.86     25000
   macro avg       0.87      0.86      0.86     25000
weighted avg       0.87      0.86      0.86     25000



In [ ]:
clf_lr.predict(tfidf.transform(["What hell was that, it's a masterpiece"]))

array(['positive'], dtype=object)